In [1]:
from applications import image_denoising
from models import ConvAutoencoderLuma
from dataloaders import SimpleLoader2d

import os
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [4]:
model = ConvAutoencoderLuma().to(device)

In [2]:
name_model = "model_1"
name_dataset = "gcg"
path_base = "/home/amedvedev/fprojects/cpp/gcg_2d_minimal/data/"

path_save_model = os.path.join("../assets/pt/", f"{name_dataset}_{name_model}.pt")

try:
    model = torch.load(path_save_model).to(device)
except Exception as e:
    print("Error when loading pretrained model. Use custom.")
    model = ConvAutoencoderLuma().to(device)

In [3]:
BATCH_SIZE = 32

path_save_train_plots = os.path.join("../assets/runs/train_plots", name_dataset, name_model)

path_train = os.path.join(path_base, name_dataset, "train")
path_test = os.path.join(path_base, name_dataset, "val")

path_train_rainy = path_train + "/noised"
path_test_rainy = path_test + "/noised"

path_train_normal = path_train + "/clear"
path_test_normal = path_test + "/clear"


width, height = 100, 100

train_noisy_loader = SimpleLoader2d(path_train_rainy, device, BATCH_SIZE, width, height)
train_normal_loader = SimpleLoader2d(path_train_normal, device, BATCH_SIZE, width, height)

val_noisy_loader =SimpleLoader2d(path_test_rainy, device, BATCH_SIZE, width, height)
val_normal_loader =SimpleLoader2d(path_test_normal, device, BATCH_SIZE, width, height)




In [4]:
def plot_step_results():
    image_denoising.save_results(model, device, path_save_train_plots, train_noisy_loader, train_normal_loader, "train",
                                 limit=1, nrow=8, op_count=1)
    image_denoising.save_results(model, device, path_save_train_plots, val_noisy_loader, val_normal_loader, "test",
                                 limit=1, nrow=8, op_count=1)
    print(f"Step results plotted to {path_save_train_plots}.")



In [15]:
EPOCHS = 5
image_denoising.train(model, train_noisy_loader, train_normal_loader, val_noisy_loader, val_normal_loader,
                      EPOCHS, device, path_save=path_save_model, callbacks=[plot_step_results])

Step results plotted to ../assets/runs/train_plots/gcg/model_1.
Training progress: |██████████████████████████████████████████████████| 100.0% Complete | 

Validating progress: |██████████████████████████████████████████████████| 100.0% Complete | 

Epoch: 1 	Training Loss: 0.004312 	Validating Loss: 0.001538 	Time: 0.57 m
Step results plotted to ../assets/runs/train_plots/gcg/model_1.


Training progress: |██████████████████████████████████████████████████| 100.0% Complete | 

Validating progress: |██████████████████████████████████████████████████| 100.0% Complete | 

Epoch: 2 	Training Loss: 0.001430 	Validating Loss: 0.001238 	Time: 0.57 m
Model saved successfully at ../assets/pt/gcg_model_1.pt.
Step results plotted to ../assets/runs/train_plots/gcg/model_1.


Training progress: |██████████████████████████████████████████████████| 100.0% Complete | 

Validating progress: |██████████████████████████████████████████████████| 100.0% Complete | 

Epoch: 3 	Training Loss: 0.001175 	Vali

In [5]:
plot_step_results()

/home/amedvedev/fprojects/python/denoising/venv/lib/python3.10/site-packages/torch/_tensor.py:836: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Step results plotted to ../assets/runs/train_plots/gcg/model_1.


In [ ]:
image_denoising.save_full_model(model, path_save_model)

In [6]:
path_save_model_onnx = os.path.join("../assets/onnx/", f"{name_dataset}_{name_model}.onnx")
inp = torch.randn((1, 1, width, height), device=device)
image_denoising.save_onnx_model(model, path_save_model_onnx, inp)

Exported graph: graph(%img : Float(1, 1, 100, 100, strides=[10000, 10000, 100, 1], requires_grad=0, device=cuda:0),
      %all_layers.0.weight : Float(76, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=1, device=cuda:0),
      %all_layers.0.bias : Float(76, strides=[1], requires_grad=1, device=cuda:0),
      %all_layers.3.weight : Float(38, 76, 2, 2, strides=[304, 4, 2, 1], requires_grad=1, device=cuda:0),
      %all_layers.3.bias : Float(38, strides=[1], requires_grad=1, device=cuda:0),
      %all_layers.6.weight : Float(38, 38, 2, 2, strides=[152, 4, 2, 1], requires_grad=1, device=cuda:0),
      %all_layers.6.bias : Float(38, strides=[1], requires_grad=1, device=cuda:0),
      %all_layers.9.weight : Float(38, 38, 2, 2, strides=[152, 4, 2, 1], requires_grad=1, device=cuda:0),
      %all_layers.9.bias : Float(38, strides=[1], requires_grad=1, device=cuda:0),
      %all_layers.11.weight : Float(38, 76, 2, 2, strides=[304, 4, 2, 1], requires_grad=1, device=cuda:0),
      %all_layers.11.bia

In [19]:
import gc

model.cpu()
del model
gc.collect()
torch.cuda.empty_cache()